In [ ]:

# import libraries
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

plt.style.use('ggplot')

import matplotlib as mpl
%matplotlib inline

from scipy.stats import chi2_contingency
from scipy.stats.contingency import association

# Read and Merge Data - basic - remove nan / duplicate values

In [ ]:

# load the data for df_final_demo
pd.set_option('display.max_columns', 120) 

df_final_demo = pd.read_csv('../Data/Cleaned/df_final_demo (1).txt')

# check for missing values
df_final_demo[df_final_demo.isnull().any(axis=1)]

# remove the missing values
df_final_demo = df_final_demo.dropna()

# remove gendr value X
df_final_demo = df_final_demo[df_final_demo['gendr'] != 'X']
df_final_demo['gendr'].value_counts()

df_final_demo


In [ ]:
# load the data for df_final_web_data

df_web_data1 = pd.read_csv('../Data/Cleaned/df_final_web_data_pt_1.txt')
df_web_data2 = pd.read_csv('../Data/Cleaned/df_final_web_data_pt_2.txt')
df_final_experiment = pd.read_csv('../Data/Cleaned/df_final_experiment_clients.txt')

# merge the data
df_web_data = pd.concat([df_web_data1, df_web_data2])

# check the duplicate
df_web_data.duplicated().sum()

# remove the duplicate
df_web_data = df_web_data.drop_duplicates()

#remove nan values for df_final_experiment
df_final_experiment = df_final_experiment.dropna()

#merge df_web_data and df_final_experiment
df_web_data = pd.merge(df_web_data, df_final_experiment, on='client_id', how='inner')

df_web_data

In [ ]:
# merge all the data
df = pd.merge(df_web_data,df_final_demo, on='client_id', how='left')

# make all columns lowercase
df.columns = df.columns.str.lower()
df.columns


#### In the dataframe df, the numerical variables are:

clnt_tenure_yr - discrete variables

clnt_tenure_mnth  - discrete variables

clnt_age - discrete variables

num_accts - discrete variables

calls_6_mnth - discrete variables

logons_6_mnth - discrete variables


bal - continuous variable 


#### The categorical variables are:

variation - nominal variable

gendr - ordinal variable

# EDA about demo dataset 


Who are the primary clients using this online process?
Are the primary clients younger or older, new or long-standing?


In [ ]:
account_plot = df['num_accts'].value_counts() \
    .head(10) \
    .plot(kind='bar', figsize=(10, 6), color='skyblue', fontsize=13, rot=45, title='Top 10 Number of Accounts')
account_plot.set_xlabel('Top 10 Number of Accounts')
account_plot.set_ylabel('Count')

In [ ]:
df['clnt_tenure_yr'].plot(kind='hist', bins=20, color='skyblue', figsize=(10, 6), fontsize=13, title='Client Tenure in Years')
plt.xlabel('Client Tenure in Years')
plt.ylabel('Count')


In [ ]:
df['clnt_tenure_yr'].plot(kind='kde', color='skyblue', figsize=(10, 6), fontsize=13, title='Client Tenure in Years')
plt.xlabel('Client Tenure in Years')
plt.ylabel('Count')

In [ ]:
# create a dataframe for the demographic data
df_demo = df[['client_id', 'date_time', 'variation', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth']]
df_demo.head(5)

# create two dataframes based on the variation
df_control = df_demo[df_demo['variation'] == 'Control']
df_test = df_demo[df_demo['variation'] == 'Test']

In [ ]:
# using stacked histogram to compare the difference between the two groups, for clnt_age and num_accts, and clin_tenure_yr, clnt_tenure_mnth

fig, axes = plt.subplots(2, 2, figsize=(20, 10))
sns.histplot(df_control['clnt_age'], kde=True, ax=axes[0, 0], color='blue', alpha=0.5)
sns.histplot(df_test['clnt_age'], kde=True, ax=axes[0, 0], color='Red', alpha=0.5)

sns.histplot(df_control['num_accts'], kde=True, ax=axes[0, 1], color='blue', alpha=0.5)
sns.histplot(df_test['num_accts'], kde=True, ax=axes[0, 1], color='red', alpha=0.5)

sns.histplot(df_control['clnt_tenure_yr'], kde=True, ax=axes[1, 0], bins=20, color='blue', alpha=0.5)
sns.histplot(df_test['clnt_tenure_yr'], kde=True, ax=axes[1, 0],bins=20, color='red', alpha=0.5)

sns.histplot(df_control['clnt_tenure_mnth'], kde=True, ax=axes[1, 1],bins=20, color='blue', alpha=0.5)
sns.histplot(df_test['clnt_tenure_mnth'], kde=True, ax=axes[1, 1],bins=20, color='red', alpha=0.5)
             
# add labels

axes[0, 0].set_xlabel('Client Age')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Client Age Distribution by Variation')

axes[0, 1].set_xlabel('Number of Accounts')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Number of Accounts Distribution by Variation')

axes[1, 0].set_xlabel('Client Tenure in Years')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Client Tenure in Years Distribution by Variation')

axes[1, 1].set_xlabel('Client Tenure in Months')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Client Tenure in Months Distribution by Variation')

# add legend

axes[0, 0].legend(['Control', 'Test'])
axes[0, 1].legend(['Control', 'Test'])
axes[1, 0].legend(['Control', 'Test'])
axes[1, 1].legend(['Control', 'Test'])


In [ ]:
df.plot(kind='scatter', x='clnt_age', y='num_accts', figsize=(10, 6), color='skyblue', fontsize=13, title='Client Age vs Number of Accounts')
plt.xlabel('Client Age')
plt.ylabel('Number of Accounts')


In [ ]:

sns.pairplot(df, 
             vars=['clnt_age', 'num_accts', 'clnt_tenure_yr', 'bal', 'calls_6_mnth', 'logons_6_mnth'], 
             diag_kind='kde', plot_kws={'color':'skyblue', 'alpha':0.5}, 
             diag_kws={'color':'skyblue', 'alpha':0.5})

In [ ]:
# use catplot to show the age distribution by variation
sns.catplot(x='gendr', y='clnt_age', hue='variation', data=df, kind='box', height=6, aspect=2)

### CLIENT Gender AND VARIANCE ANALYSIS

In [ ]:
# create a grouped bar chart to compare gender distribution by variation

sns.set_theme(style="whitegrid", palette='viridis')
sns.catplot(x='gendr', kind='count', hue='variation', data=df_demo, height=5, aspect=2)

# add labels
plt.xlabel('Gender')
plt.ylabel('Count')
# make visual tight
plt.tight_layout()

# add title
plt.title('The Gender Distribution by Variation')

# add legend
plt.legend(title='Variation', loc='upper right', labels=['Control', 'Test'])

In [ ]:
# create a cross tab for the gender and variation
gendr_variation = pd.crosstab(df_demo['gendr'], df_demo['variation'])

# compute the probability of the age and variation
gendr_variation['prob_control'] = gendr_variation['Control'] / gendr_variation['Control'].sum()
gendr_variation['prob_test'] = gendr_variation['Test'] / gendr_variation['Test'].sum()

# use chi-square test to determine if the age and variation are independent
from scipy.stats import chi2_contingency
chi2, p, dof, ex = chi2_contingency(gendr_variation[['Control', 'Test']])
p


In [ ]:
# explain the result
if p < 0.05:
    print('There is a significant difference between the gender and variation')
else:
    print('There is no significant difference between the gender and variation')

In [ ]:
# calls_6_mnth: Records the number of times the client reached out over a call in the past six months.
# logons_6_mnth: Reflects the frequency with which the client logged onto Vanguard’s platform over the last six months.

#using displot to compare the difference between the two groups, for calls_6_mnth and logons_6_mnth
fig, axes = plt.subplots(1, 2, figsize=(20, 5))
sns.histplot(df_control['calls_6_mnth'], kde=True, ax=axes[0], color='blue', alpha=0.5)
sns.histplot(df_test['calls_6_mnth'], kde=True, ax=axes[0], color='red', alpha=0.5)

sns.histplot(df_control['logons_6_mnth'], kde=True, ax=axes[1], color='blue', alpha=0.5)
sns.histplot(df_test['logons_6_mnth'], kde=True, ax=axes[1], color='red', alpha=0.5)

# add labels
axes[0].set_xlabel('Number of Calls in the Last 6 Months')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Number of Calls in the Last 6 Months Distribution by Variation')

axes[1].set_xlabel('Number of Logons in the Last 6 Months')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Number of Logons in the Last 6 Months Distribution by Variation')

# add legend
axes[0].legend(['Control', 'Test'])
axes[1].legend(['Control', 'Test'])


In [ ]:
# pie chart to show the distribution of the variation
plt.figure(figsize=(10, 7))
df_demo['variation'].value_counts(dropna=False).plot.pie(autopct='%1.1f%%',colors=['#FFA07A', '#20B2AA', '#87CEFA'])
plt.title('Variation Distribution')
plt.show()

## Checking the correlation between numerical variables

In [ ]:
cor_numerical = df_demo[['clnt_tenure_yr', 'clnt_age', 'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth']].corr()

In [ ]:
# use heatmap to show the correlation between the numerical variables, clnt_tenure_yr, clnt_tenure_mnth, clnt_age, num_accts, bal, calls_6_mnth, logons_6_mnth


sns.heatmap(cor_numerical, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')

# add labels
plt.xlabel('Numerical Variables')
plt.ylabel('Numerical Variables')


### Explain the result
The heatmap shows that there is a strong correlation between the number of accounts and the balance.

There is also a moderate correlation between the number of calls in the last 6 months and the number of logons in the last 6 months.


The heatmap shows the correlation between the numerical variables. The correlation ranges from -1 to 1.


A correlation of 1 indicates a perfect positive correlation, while a correlation of -1 indicates a perfect negative correlation.



In [ ]:
df_demo['num_accts'].unique()

df_demo.isnull().sum()

# Remove rows with NaN values
df_demo_clean = df_demo.dropna()

# Calculate Pearson correlation
corr, p = pearsonr(df_demo_clean['num_accts'], df_demo_clean['bal'])
corr, p

# explain the pearson value
if corr > 0.5:
    print('The correlation between the number of accounts and balance is strong')
elif corr > 0.3:
    print('The correlation between the number of accounts and balance is moderate')
else:
    print('The correlation between the number of accounts and balance is weak')
    

In [ ]:
# use QQ plot to check the normality of the num_accts and bal
import statsmodels.api as sm

sm.qqplot(df_demo['num_accts'], line='s')
plt.title('QQ Plot - num_accts')
plt.show()

# QQ plot for bal
sm.qqplot(df['bal'], line='s')
plt.title('QQ Plot - bal')
plt.show()


# KPIS

## Caculate the completion rate by counting how many confirm by client_id



### The defination of completion rate:  When a client has reach to confirm step which is process_step is confrim, meaning the client has complete the process, only client only has one confirm 

In [ ]:

# add a new column to show the number of steps
df['num_steps'] = df['process_step'].map({'start':0, 'step_1':1, 'step_2':2, 'step_3':3, 'confirm':4})

# df sorted by client_id and date
df = df.sort_values(by=['client_id', 'date_time'])

In [ ]:
# count the number of steps for each client
df_count_step = df.groupby(['client_id', 'num_steps']).size().unstack(fill_value=0)
# merge the data with the demographic data
df_num_steps = pd.merge(df_count_step, df_final_experiment, on='client_id', how='left')


# if a client_id, cloumn 4 value >=1 then the client has completed the process
df_num_steps['completed'] = df_num_steps[4] >= 1
df_num_steps['completed'].value_counts()

# calculate the percentage of clients who completed the process
completed_clients = df_num_steps[df_num_steps['completed'] == True]
not_completed_clients = df_num_steps[df_num_steps['completed'] == False]

# calculate the percentage of clients who completed the process 
completed_percentage = (completed_clients['client_id'].nunique() / df['client_id'].nunique()) * 100

# calculate the percentage of clients who did not complete the process
not_completed_percentage = (not_completed_clients['client_id'].nunique() / df['client_id'].nunique()) * 100

completed_percentage, not_completed_percentage

In [ ]:
df_num_steps

In [ ]:
# using grouped bar chart to completed precentage by variation
sns.set_theme(style="whitegrid", palette='viridis')
sns.catplot(x='completed', kind='count', hue='Variation', data=df_num_steps, height=5, aspect=2)

# add labels
plt.xlabel('Completed')
plt.ylabel('Count')
# make visual tight
plt.tight_layout()

# add title
plt.title('The Completed Percentage by Variation')

In [ ]:
# create a cross tab for the completed and variation
completed_variation = pd.crosstab(df_num_steps['completed'], df_num_steps['Variation'])
# add proportion of the completed and variation
completed_variation['prob_control'] = completed_variation['Control'] / completed_variation['Control'].sum()
completed_variation['prob_test'] = completed_variation['Test'] / completed_variation['Test'].sum()


In [ ]:
completed_variation

# make pie charts to compare the percentage of clients who completed the process by control and test group
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
completed_variation['Control'].plot.pie(autopct='%1.1f%%', ax=axes[0], colors=['#FFA07A', '#20B2AA', '#87CEFA'])
axes[0].set_title('Percentage of Clients Who Completed the Process in the Control Group')

completed_variation['Test'].plot.pie(autopct='%1.1f%%', ax=axes[1], colors=['#FFA07A', '#20B2AA', '#87CEFA'])
axes[1].set_title('Percentage of Clients Who Completed the Process in the Test Group')

# add legend
axes[0].legend(['Control', 'Test'])
axes[1].legend(['Control', 'Test'])



## ADD THE TIME SPEND ON EACH STEP

In [ ]:

# organzie the data by date_time
df['date_time'] = pd.to_datetime(df['date_time'])
#calculate the time difference by client_id
df['time_diff'] = np.where(df['visit_id'] != df['visit_id'].shift(1), 0, df.sort_values(['client_id', 'date_time']).groupby('client_id')['date_time'].diff().dt.total_seconds().fillna(0))

# add time spend column which is time_diff move up by one row
df['time_spend'] = df.groupby(["visit_id"])['time_diff'].shift(-1)

In [ ]:
pd.set_option('display.max_rows', 120)
# df[df['client_id'] == 3825423]

In [ ]:
# each client the average time spend on each step by variation
df_web_data_grouped_client = df.groupby(['variation', 'client_id', 'num_steps'])['time_spend'].mean().reset_index()
df_web_data_grouped_client

# using seaborn catplot to create a grouped bar chart
sns.catplot(x='num_steps', y='time_spend', hue='variation', data=df_web_data_grouped, kind='bar', height=6, aspect=2, palette = 'viridis')
plt.title('Time Spend on Each Step by Variation')
plt.xticks([0, 1, 2,3, 4], ['start','step 1', 'step 2', 'step 3', 'confirm'])
plt.show()


In [ ]:
# calculate the average time spend on each step by variation
df_web_data_grouped = df.groupby(['variation', 'num_steps'])['time_spend'].mean().reset_index()
df_web_data_grouped

# Mark the error

In [ ]:
df_num_steps

In [ ]:
# add the error column to the data
df['error'] = np.where((df['visit_id'] == df['visit_id'].shift(1)) & (df['num_steps'] <= df['num_steps'].shift(1)), 'error', 'nan')

In [ ]:
# checking this id and sort it by date_time 
# df[df['client_id'] == 8290360]

In [ ]:
pd.set_option('display.max_rows', 120)
 # df[df['client_id'] == 1336]

In [ ]:
# show a countplot of the error column by variation
sns.countplot(x='error', data=df, hue='variation', palette='viridis')

In [ ]:
# show error by num_steps and variation
df_web_data_error = df[df['error'] == 'error']
df_web_data_error_grouped = df_web_data_error.groupby(['variation', 'num_steps'])['error'].count().reset_index()
df_web_data_error_grouped

In [ ]:
# using seaborn catplot to create a grouped bar chart
sns.catplot(x='num_steps', y='error', hue='variation', data=df_web_data_error_grouped, kind='bar', height=6, aspect=2, palette = 'viridis')
plt.title('Error by Step and Variation')
plt.xticks([0, 1, 2,3, 4], ['start','step 1', 'step 2', 'step 3', 'confirm'])
plt.show()

# Hypothesis Testing

Given the data and KPIs you have explored discussed, one interesting hypothesis to test is related to the completion rate between the Test and Control groups. 

Since the new design (Test group) had a higher completion rate compared to the old design (Control group), you are required to confirm if this difference is statistically significant.

Make sure to define the proper null and an alternative hypothesis to test it. 

Use the provided data to test these hypotheses, and determine if you can reject the null hypothesis in favor of the alternative. 

Make sure to consider the significance level, p-value, the statistical test prerequisites, and other relevant statistical measures in your analysis.

In [ ]:
df_num_steps

In [ ]:
df_test = df_num_steps[df_num_steps["Variation"]=="Test"]["completed"]
df_control = df_num_steps[df_num_steps["Variation"]=="Control"]["completed"]

In [ ]:
#Set the hypothesis

#H0: Test Group completion rate <= Control Group completion rate
#H1: Test Group completion rate > Control

#significance level = 0.05

In [ ]:
import scipy.stats as st
st.ttest_ind(df_test,df_control, equal_var='less')
t_stat, p_value = st.ttest_ind(df_test, df_control, equal_var=False)
t_stat, p_value


Because p_value is lower than significance level, we reject the null hypothesis, that means Test Group Completion rate is higher than Control

The introduction of a new UI design comes with its associated costs: design, development, testing, potential training for staff, and possible short-term disruptions or adjustments for users. To justify these costs, Vanguard has determined that any new design should lead to a minimum increase in the completion rate to be deemed cost-effective.

Threshold: Vanguard has set this minimum increase in completion rate at 5%. This is the rate at which the projected benefits, in terms of increased user engagement and potential revenue, are estimated to outweigh the costs of the new design.

You are required to carry out another analysis, ensuring that the observed increase in completion rate from the A/B test meets or exceeds this 5% threshold. If the new design doesn’t lead to at least this level of improvement, it may not be justifiable from a cost perspective, regardless of its statistical significance.

In [ ]:
# A/B testing to compare the completion rate between the control and test group 

# set the hypothesis
# H0: Test Group completion rate <= Control Group completion rate  = 0.05
# H1: Test Group completion rate > Control Group completion rate   = 0.05

# significance level = 0.05
import scipy.stats as st

# calculate the completion rate for the control and test group
control_completion_rate = df_num_steps[df_num_steps['Variation'] == 'Control']['completed'].mean()
test_completion_rate = df_num_steps[df_num_steps['Variation'] == 'Test']['completed'].mean()

# calculate the completion rate difference
completion_rate_diff = test_completion_rate - control_completion_rate

# calculate the standard deviation for the control and test group
control_std = df_num_steps[df_num_steps['Variation'] == 'Control']['completed'].std()
test_std = df_num_steps[df_num_steps['Variation'] == 'Test']['completed'].std()

# calculate the standard error
control_se = control_std / np.sqrt(df_num_steps[df_num_steps['Variation'] == 'Control']['completed'].count())
test_se = test_std / np.sqrt(df_num_steps[df_num_steps['Variation'] == 'Test']['completed'].count())

# calculate the t-statistic
t_stat = (test_completion_rate - control_completion_rate) / np.sqrt(control_se**2 + test_se**2)
t_stat

# calculate the degrees of freedom
df1 = df_num_steps[df_num_steps['Variation'] == 'Control']['completed'].count() + df_num_steps[df_num_steps['Variation'] == 'Test']['completed'].count() - 2

# calculate the p-value
p_value = 1 - st.t.cdf(t_stat, df1)
p_value

# explain the result
if p_value < 0.05:
    print('There is a significant difference between the completion rate of the control and test group')
else:
    print('There is no significant difference between the completion rate of the control and test group')



In [ ]:
control_completion_rate 

Design Effectiveness
Was the experiment well-structured?

Were clients randomly and equally divided between the old and new designs?

Were there any biases?

Duration Assessment
Was the timeframe of the experiment (from 3/15/2017 to 6/20/2017) adequate to gather meaningful data and insights?

Additional Data Needs
What other data, if available, could enhance the analysis?




In [ ]:
df_num_steps

In [ ]:
# set a date range for the data, start date os 3/15/2017 and end date is 6/20/2017

start_date = '2017-03-15'
end_date = '2017-06-20'

# only select the data within the date range
df_date = df[(df['date_time'] >= start_date) & (df['date_time'] <= end_date)]
